In [ ]:
import cv2
import json
import pandas as pd
import os
import numpy as np
import csv
from ast import literal_eval
from shapely.geometry import Polygon

names = os.listdir('runs\\detect')[:]
names2 = []

# Сортируем файлы predict
for name in names:
    if name[7:] != '':
        names2.append(int(name[7:]))
    else:
        names2.append(0)

names2.sort()

for i, num in enumerate(names2):
    if num != 0:
        names[i] = 'predict' + str(num)
    else:
        names[i] = 'predict'
X, Y = 1920, 1280

filenames = os.listdir('test\\Test_Newest_2')

def area_interest():
    a = []
    for txt in text['areas']:
        for j in txt:
            a.append([j[0] * X, j[1] * Y])
    leftmost_point, *_, rightmost_point = sorted(a, key=lambda lst: lst[0])
    x1, y1 = leftmost_point
    x2, y2 = rightmost_point
    border_slope = (y2 - y1) / (x2 - x1)
    
    points_below_border = [(x1, y1)]
    points_above_border = [(x2, y2)]
    
    for x, y in a:
        border = border_slope * (x - x1) + y1
        if y < border:
            points_below_border.append((x, y))
        elif y > border:
            points_above_border.append((x, y))
    
    points_below_border.sort()
    points_above_border.sort(reverse=True)
    
    points_above_border.append((x1, y1))
    
    merged_points = points_below_border + points_above_border
    return merged_points

def PolyArea(x, y):  # x и y - списки соответствующих координат
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))


path1 = 'runs\\detect\\'
path2 = 'test\\Test_markup'

In [ ]:
with open('track_predict.csv', 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["file_name","quantity_car","average_speed_car","quantity_van","average_speed_van","quantity_bus","average_speed_bus"])
    for i1 in range(len(names)):
        with open(os.path.join(path2, filenames[i][:-3] + 'json'), 'r', encoding='utf-8') as f:
            text = json.load(f)
        all_data = []
        alldata = pd.DataFrame()
        clipping_polygon = np.array(area_interest())
        clipped_polygon = []
        speeds = 0
        ls = 0
        for (root, dirs, file) in os.walk(path1 + names[i]):
            predid = pd.DataFrame([[0, 0, 0, 0, 0, 0]], columns = ['class', 'x1', 'y1', 'x2', 'y2', 'id'])
            predcentroid = False
            for f in file:


                data = pd.read_csv(root + '/' + f, sep=' ', header=None)
                if data.shape[1] != 6:
                    ls += 1
                    continue
                data.columns = ['class', 'x1', 'y1', 'x2', 'y2', 'id']
                if len(alldata) == 0:
                    alldata = data
                else:
                    alldata = pd.concat([alldata, data])
                for j in range(len(data)):
                    x1, y1, x2, y2 = data.iloc[j][['x1', 'y1', 'x2', 'y2']]
                    x1, x2 = x1 * X, (x2 * X) / 2
                    y1, y2 = y1 * Y, (y2 * Y) / 2
                    subject_polygon = np.array([[x1 - x2, y1 - y2], [x1 + x2, y1- y2], [x1 + x2, y1 +y2], [x1 - x2, y1 +y2], [x1 - x2, y1- y2]])
                    Inter = Polygon(subject_polygon).intersection(Polygon(clipping_polygon))
                    clipped_polygon = Inter
                    if Inter:
                        if clipped_polygon.geom_type == 'MultiPolygon':
                            for z in clipped_polygon.geoms:
                                x, y = z.exterior.xy
                        else:
                            x, y = clipped_polygon.exterior.xy
                        if round(PolyArea(x, y) / PolyArea(subject_polygon[:, 0], subject_polygon[:, 1]), 4) * 100 > 1:
                            centroid = (sum(subject_polygon[:, 0]) / len(subject_polygon), sum(subject_polygon[:, 1]) / len(subject_polygon))
                            all_data.append([data.iloc[j]['class'], data.iloc[j]['id'], centroid])

        all_data = pd.DataFrame(all_data, columns=['class', 'id', 'centr'])                
        all1 = all_data[all_data.groupby('id').id.transform(len) > 1]
        all1 = all1.drop_duplicates('id')
        sp1 = pd.DataFrame(all1.groupby('class').count()['id'])

        sp2 = []
        speeds = []
        for j in all_data['class'].unique():
            for i in all_data['id'].unique():
                now = all_data[(all_data['id'] == i) & (all_data['class'] == j)]
                if len(now) <= 1:
                    continue
                if len(now) > 5:
                    speeds.append(20 / len(now) * 3.6 * 5)

            sp2.append([j, sum(speeds) / len(speeds)])

        sp2 = pd.DataFrame(sp2, columns=['class', 'val'])
        sp1.reset_index()
        sp1['class'] = sp1.index
        sp2.reset_index()
        sp3 = pd.DataFrame([], columns=['class', 'count', 'speed'])
        sp3['class'] = [2, 7, 5]
        if 2.0 not in sp1['class']:
            sp1.loc[len(sp1.index)] = [0, 0]
        if 7.0 not in sp1['class']:
            sp1.loc[len(sp1.index)] = [0, 0]
        if 5.0 not in sp1['class']:
            sp1.loc[len(sp1.index)] = [0, 0]

        if 2.0 not in sp2['class']:
            sp2.loc[len(sp2.index)] = [0, 0]
        if 7.0 not in sp2['class']:
            sp2.loc[len(sp2.index)] = [0, 0]
        if 5.0 not in sp2['class']:
            sp2.loc[len(sp2.index)] = [0, 0]

        sp3['count'] = [int(sp1[sp1['class'] == 2]['id']), int(sp1[sp1['class'] == 7]['id']), int(sp1[sp1['class'] == 5]['id'])]
        sp3['speed'] = [float(sp2[sp2['class'] == 2]['val']), float(sp2[sp2['class'] == 7]['val']), float(sp2[sp2['class'] == 5]['val'])]
        spall = []
        for j in range(3):
            spall.append(sp3.iloc[j]['class'])
            spall.append(sp3.iloc[j]['count'])
            spall.append(sp3.iloc[j]['speed'])

        print(spall)

        writer.writerow([int(i1), spall[1], spall[2], spall[4], spall[5], spall[7], spall[8]])
        csv_file.flush()